Notes: how will the heuristic value interact with the return values from minimax? Need to use + and - inf to stop it from prioritizing heuristic

Create a fake game instance to test alphabeta pruning - just needs to return legal moves

Have disabled random.shuffle in isolation.py!

self.score is not implemented

why is the search exception getting called?

In [28]:
from pdb import set_trace
import timeit

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [61]:
from isolation import Board
import copy

class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    pass

def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    
    if game.is_loser(player):
        return float('-inf')

    if game.is_winner(player):
        return float('inf')
    
    # Centre bias
    centre = (int(game.width/2),int(game.height/2))
    
    player_location = game.get_player_location(player)
    opponent_location = game.get_player_location(game.get_opponent(player))
    
    player_centre_diff = abs(player_location[0] - centre[0]) * abs(player_location[1] - centre[1])
    opponent_centre_diff = abs(opponent_location[0] - centre[0]) * abs(opponent_location[1] - centre[1])
    
    centrification = opponent_centre_diff - player_centre_diff
        
    return float(centrification)


def custom_score_2(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    if game.is_loser(player):
        return float('-inf')

    if game.is_winner(player):
        return float('inf')
    
    player_location = game.get_player_location(player)
    opponent_location = game.get_player_location(game.get_opponent(player))
    
    # Aggressiveness
    aggressiveness = abs(player_location[0] - opponent_location[0]) * abs(player_location[1] - opponent_location[1])
    
    return float(aggressiveness)

def custom_score_3(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """

    if game.is_loser(player):
        return float('-inf')

    if game.is_winner(player):
        return float('inf')
    
    # Centre bias
    centre = (int(game.width/2),int(game.height/2))
    
    player_location = game.get_player_location(player)
    opponent_location = game.get_player_location(game.get_opponent(player))
    
    player_centre_diff = abs(player_location[0] - centre[0]) * abs(player_location[1] - centre[1])
    opponent_centre_diff = abs(opponent_location[0] - centre[0]) * abs(opponent_location[1] - centre[1])
    
    centrification = opponent_centre_diff - player_centre_diff
    
    # Aggressiveness
    aggressiveness = abs(player_location[0] - opponent_location[0]) * abs(player_location[1] - opponent_location[1])
    
    return float(centrification * aggressiveness)

class IsolationPlayer:
    """Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.

    ********************  DO NOT MODIFY THIS CLASS  ********************

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=3, score_fn=custom_score, timeout=10.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.    
            best_move = self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass
                # CH: what to do here?

        return best_move
   
    def min_value(self, game, depth):
        """ Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
            
        if depth <= 0:
            return self.score(game, self)

        val = 1
        for move in game.get_legal_moves():
            val = min(val, self.max_value(game.forecast_move(move), depth-1))
        return val

    def max_value(self, game, depth):
        """ Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
            
        if depth <= 0:
            return self.score(game, self)
            
        val = -1
        for move in game.get_legal_moves():
            val = max(val, self.min_value(game.forecast_move(move), depth-1))
        return val


    def minimax(self, game, depth):
        """Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
        
        
        
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        # TODO: finish this function!
        best_score = float('-inf')
        best_move = (-1,-1)
        
        for move in game.get_legal_moves():
            move_score = self.min_value(game.forecast_move(move), depth - 1)
            if move_score > best_score:
                best_score = move_score
                best_move = move
        return best_move


class AlphaBetaPlayer(IsolationPlayer):

    def get_move(self, game, time_left):
       
        self.time_left = time_left

        best_move = (-1,-1)
        
        try:
            depth = 1
            while True: 
                best_move = self.alphabeta(game, depth)
                depth += 1

        except SearchTimeout:
            return best_move
        
        finally:
            return best_move


    def terminal_test(self, game, depth):
        return depth <= 0 or game.get_legal_moves() == 0


    def max_value(self, game, depth, alpha, beta):
     
        if self.terminal_test(game, depth):
            return self.score(game, self)

        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        v = float('-inf')
        for m in game.get_legal_moves():
            v = max(v, self.min_value(game.forecast_move(m), depth - 1, alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(self, game, depth, alpha, beta):
        if self.terminal_test(game, depth):
            return self.score(game, self)

        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        v = float('inf')
        for m in game.get_legal_moves():
           # the depth should be decremented by 1 on each call
            v = min(v, self.max_value(game.forecast_move(m), depth - 1, alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    def alphabeta(self, game, depth, alpha=float('-inf'), beta=float('inf')):
       
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()

        legal_moves = game.get_legal_moves()
        if not legal_moves:
            return (-1,-1)

        best_score = float('-inf')
        best_move = legal_moves[0]

        for m in legal_moves:
            v = self.min_value(game.forecast_move(m), depth-1, best_score, beta)
            if v > best_score:
                best_score = v
                best_move = m

        return best_move



In [59]:
one = AlphaBetaPlayer(3, custom_score)
two = copy.deepcopy(one)
small_board = Board(one, two,3,3)
small_board.play()
print(small_board.to_string())

(<__main__.AlphaBetaPlayer at 0x1cbb868c080>, [], 'forfeit')

     0   1   2
0  |   |   |   | 
1  |   |   |   | 
2  |   |   |   | 



In [62]:
# For Minimax, 3 is the maximum depth before timemouts too common
# Alphabeta can handle 10
one = MinimaxPlayer(2, custom_score_3)
two = AlphaBetaPlayer(8, custom_score)
small_board = Board(one, two,7,7)
small_board.play()
print(small_board.to_string())

(<__main__.AlphaBetaPlayer at 0x1cbb8642390>,
 [[3, 0],
  [0, 0],
  [2, 2],
  [1, 2],
  [3, 4],
  [3, 1],
  [1, 3],
  [5, 2],
  [3, 2],
  [4, 0],
  [4, 4],
  [2, 1],
  [3, 6],
  [3, 3],
  [2, 4],
  [4, 5],
  [0, 3],
  [5, 3],
  [1, 1],
  [6, 5],
  [2, 3],
  [4, 6],
  [4, 2],
  [5, 4],
  [6, 3],
  [3, 5]],
 'forfeit')

     0   1   2   3   4   5   6
0  | - |   |   | - |   |   |   | 
1  |   | - | - | - |   |   |   | 
2  |   | - | - | - | - |   |   | 
3  | - | - | - | - | - | 2 | - | 
4  | - |   | - |   | - | - | - | 
5  |   |   | - | - | - |   |   | 
6  |   |   |   | 1 |   | - |   | 



In [37]:
time_limit = 10**200

def time_method(method):

    def timed(*args, **kw):
        ts = timeit.default_timer() * 1000
        result = method(*args, **kw)
        te = timeit.default_timer() * 1000

        print('%r (%r, %r) %2.2f' % (method.__name__, args, kw, te-ts))
        return result

    return timed

@time_method
def minimax_test():
    one = MinimaxPlayer(5, custom_score)
    two = copy.deepcopy(one)
    small_board = Board(one, two, 7,7)
    result = small_board.play(time_limit)
    print(small_board.to_string())
    return result
    
@time_method
def alphabeta_test():
    one = AlphaBetaPlayer(5, custom_score)
    two = copy.deepcopy(one)
    small_board = Board(one, two, 7,7)
    result = small_board.play(time_limit)
    print(small_board.to_string())
    return result

In [38]:
minimax_test()

TypeError: unsupported operand type(s) for /: 'method' and 'int'

In [ ]:
alphabeta_test()